In [1]:
import pandas as pd
import numpy as np
import json

# Replace double quotes with single quotes for moods JSON arrays
f = open('song_data.csv','r+')
text = f.read()
text = text.replace('"["', '"[?').replace('", "', '?,?').replace('"]"', '?]"')
f.seek(0)
f.write(text)
f.close()

# Importing data
df = pd.read_csv('song_data.csv', index_col=0)
for col in df.columns:
    if col=='uuid' or 'gyro' in col or 'accel' in col:
        df.drop(col, axis=1, inplace=True)
    elif col not in ['id','isSkipped','activity']:
        if col == 'moods':
            df[col] = df[col].apply(lambda x:x.replace('?','"'))
        df[col] = df[col].apply(json.loads)
df.head()

,optical,temp,humidity,moods,isSkipped
id,,,,,
1,"[139.64, 138.36, 139.64, 140.28]","[30.50567626953125, 30.50567626953125, 30.5056...","[71.3134765625, 71.3134765625, 71.3134765625]","[Depressive, Atmospheric]",0
2,"[123.24, 123.24, 139.32, 228.64]","[31.69403076171875, 31.69403076171875, 31.6940...","[67.05322265625, 67.05322265625, 67.0532226562...",[Depressive],0
3,"[256.08, 307.84000000000003, 315.2, 301.36, 30...","[32.21771240234375, 32.21771240234375, 32.2177...","[65.850830078125, 65.850830078125, 65.85083007...","[Passionate, Depressive]",0
4,"[127.08, 126.76, 125.48, 124.52]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...",[Elegant],1
5,"[145.76, 144.48, 146.4, 144.8]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...","[Passionate, Depressive]",1


In [2]:
# Filtering defective data

defective_ids = []
for id,row in df.iterrows():
    # defective if temp array only has -40 values
    # defective if any humidity values are above 99.99
    if len([k for k in row['temp'] if k==-40]) == len(row['temp']) or \
       len([k for k in row['humidity'] if k>99.99]) == len(row['humidity']):
        defective_ids.append(id)
    
    # remove -40 temp values and >99.99 humidity values
    elif (-40 in row['temp']) or len([k for k in row['humidity'] if k>99.99]):
        row['temp'] = [k for k in row['temp'] if k!=-40]
        row['humidity'] = [k for k in row['humidity'] if k<99.99]

filtered_df = df[~df.index.isin(defective_ids)].copy() # .copy() to avoid warning
print('Defective row ids are: ',defective_ids)

Defective row ids are:  [21, 22, 33, 214, 236, 238, 245, 246, 247, 248, 249, 250, 251, 252]


In [3]:
# Obtain mean optical, temp and humidity values

for col in df.columns:
    if col not in ['moods','isSkipped']:
        filtered_df[col] = filtered_df[col].apply(np.mean)
filtered_df.head()

,optical,temp,humidity,moods,isSkipped
id,,,,,
1,139.480,30.505676,71.313477,"[Depressive, Atmospheric]",0
2,153.610,31.694031,67.053223,[Depressive],0
3,297.792,32.217712,65.850830,"[Passionate, Depressive]",0
4,125.960,32.429199,64.672852,[Elegant],1
5,145.360,32.429199,64.672852,"[Passionate, Depressive]",1


In [5]:
# One-hot encoding for moods

moods = []
for k in filtered_df['moods'].values:
    moods += list(k)
moods = np.unique(np.array(moods))
for mood in moods:
    mood_values = filtered_df['moods'].astype(str).str.contains(mood)
    filtered_df[mood] = mood_values
filtered_df.drop('moods', axis=1, inplace=True)
filtered_df.head()

,optical,temp,humidity,isSkipped,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
1,139.480,30.505676,71.313477,0,False,False,True,False,True,False,False,False
2,153.610,31.694031,67.053223,0,False,False,False,False,True,False,False,False
3,297.792,32.217712,65.850830,0,False,False,False,False,True,False,True,False
4,125.960,32.429199,64.672852,1,False,False,False,False,False,True,False,False
5,145.360,32.429199,64.672852,1,False,False,False,False,True,False,True,False
